In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os
from functools import partial

from dotenv import load_dotenv

from optimization import jg_opt
from pisa.administrative_area import AdministrativeArea
from pisa.facilities import Facilities
from pisa.population import WorldpopPopulation
from pisa.population_served_by_isopolygons import get_population_served_by_isopolygons

# keep the API keys in a `.env` file in the local root directory
load_dotenv()


True

Some choices:

- Population with WorldPop or Facebook
- isopolygon calculator either Mapbox or OSM

### Define Administrative Area

Let op: the administrative area is a country because of the package that we use for fetching the data (gadm).

The naming is confusing: we want the administrative area to be a subset of a country (in this case, the administrative area should be Baucau). 

Should be easy to fix once we change gadm to pygadm (see issue #59 on GitHub)

In [15]:
timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

INFO:pisa.administrative_area:Validating country name: Timor-Leste
INFO:pisa.administrative_area:Country name 'Timor-Leste' validated successfully
INFO:pisa.administrative_area:Retrieving boundaries of all administrative areas of level 1 for country Timor-Leste


### Get population 
In this example, from WorldPop

In [16]:
population_gdf = WorldpopPopulation(
    admin_area_boundaries=baucau, iso3_country_code=timor_leste.get_iso3_country_code()
).get_population_gdf()

population_count = population_gdf.population.values

### Create instance of Facilities class for Baucau

In [17]:
facilities_calculator = Facilities(admin_area_boundaries=baucau)

## Calculate isopolygons

In [18]:
DISTANCE_TYPE = "length"

DISTANCE_VALUES = [2000, 5000, 10000]

MODE_OF_TRANSPORT = "driving"

#### Using OSM alternative calculator

In [ ]:
from pisa.osm_road_network import OsmRoadNetwork

road_network = OsmRoadNetwork(
    admin_area_boundaries=baucau,
    mode_of_transport=MODE_OF_TRANSPORT,
    distance_type=DISTANCE_TYPE,
).get_osm_road_network()

INFO:pisa.osm_road_network:OSM road network set with parameters network_type 'drive' and distance_type 'length'


#### Calculate isopolygons for existing facilities

In [32]:
from pisa.isopolygons import OsmIsopolygonCalculatorAlternative

isopolygons_existing_facilities = OsmIsopolygonCalculatorAlternative(
    facilities_df=facilities_calculator.get_existing_facilities(),
    distance_type=DISTANCE_TYPE,
    distance_values=DISTANCE_VALUES,
    road_network=road_network,
).calculate_isopolygons()

population_served_current = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_existing_facilities
)

current = {DISTANCE_TYPE: population_served_current}

INFO:pisa.facilities:Retrieving existing facilities with tags {'amenity': 'hospital', 'building': 'hospital'} using OSM.
INFO:pisa.facilities:Successfully retrieved existing facilities from OSM.


#### Calculate isopolygons for potential facilities

In [33]:
isopolygons_potential_facilities = OsmIsopolygonCalculatorAlternative(
    facilities_df=facilities_calculator.estimate_potential_facilities(),
    distance_type=DISTANCE_TYPE,
    distance_values=DISTANCE_VALUES,
    road_network=road_network,
).calculate_isopolygons()

population_served_potential = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_potential_facilities
)

potential = {DISTANCE_TYPE: population_served_potential}

### Calculate solutions

In [34]:
BUDGET = [
    5,
    20,
    50,
]  # budget for the optimization in terms of how many locations can be built


CBC_SOLVER_PATH = os.getenv(
    "CBC_SOLVER_PATH"
)  # path to the cbc executable (e.g. /opt/homebrew/bin/cbc)

cbc_optimize = partial(jg_opt.OpenOptimize, solver_path=CBC_SOLVER_PATH)

values, solutions = jg_opt.Solve(
    household=population_count,
    current=current,
    potential=potential,
    accessibility=DISTANCE_TYPE,
    budgets=BUDGET,
    optimize=cbc_optimize,
    type="ID",
)

In [35]:
values

,10000,5000,2000
5,0.263659,0.205705,0.123098
20,0.267482,0.221113,0.131744
50,0.267482,0.224543,0.133409


In [36]:
solutions

,10000,5000,2000
5,"[25, 27, 22, 18, 38]","[25, 27, 22, 43, 9]","[39, 43, 25, 22, 7]"
20,"[25, 27, 22, 18, 38, 14, 37, 43, 35, 0, 3, 9, ...","[25, 27, 22, 43, 9, 16, 18, 42, 38, 39, 28, 12...","[39, 43, 25, 22, 7, 27, 24, 32, 38, 10, 16, 42..."
50,"[25, 27, 22, 18, 38, 14, 37, 43, 35, 0, 3, 9, ...","[25, 27, 22, 43, 9, 16, 18, 42, 38, 39, 28, 12...","[39, 43, 25, 22, 7, 27, 24, 32, 38, 10, 16, 42..."
